In [72]:
!pip install tableone

In [73]:
# replace protean-chassis-368116 with your own project id
%env GOOGLE_CLOUD_PROJECT = protean-chassis-368116

import matplotlib.pyplot as plt
import pandas as pd
from google.colab import files
from google.cloud import bigquery
from tableone import TableOne
import missingno as msno
import matplotlib.pyplot as plt
import numpy as np

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

import warnings
warnings.filterwarnings("ignore")

%load_ext google.colab.data_table

mimic_iv = {}
dataframes = {}

env: GOOGLE_CLOUD_PROJECT=protean-chassis-368116
Authenticated
The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [74]:
def get_vars_df(sheet_id, gid):

  return pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid={gid}')

## get data

### 1. original

In [75]:
%%bigquery mimic_iv_pat

SELECT DISTINCT
    icu.subject_id
  , icu.hadm_id
  , icu.stay_id
  , CASE WHEN icu.gender = "F" THEN 1 ELSE 0 END AS sex_female
  , icu.race AS race_ethnicity
  , icu.admission_age

-- ICU stays
FROM physionet-data.mimiciv_derived.icustay_detail
AS icu

Query is running:   0%|          |

Downloading:   0%|          |

In [76]:
sheets_id = "1Hv_sOd0--6TPYiB3Crjdn_JrIhIazXXJc05mL4GefOU"
gid = "1877090341"

mimic_iv_map = get_vars_df(sheets_id, gid)

mapping_series = mimic_iv_map.set_index('original')['mapping']

# Map the 'race_ethnicity' column in the 'mimic_iv_pat' DataFrame
mimic_iv_pat['race_ethnicity'] = mimic_iv_pat['race_ethnicity'].map(mapping_series)

mimic_iv_pat = mimic_iv_pat.groupby('subject_id').first()

In [77]:
dataframes[1] = mimic_iv_pat[['race_ethnicity', 'sex_female', 'admission_age']]

### 2. abgs

In [78]:
%%bigquery mimic_iv_abgs

SELECT *
FROM
(
  SELECT *
  FROM (
    SELECT subject_id, hadm_id, charttime, label, value
    FROM
    (
      -- subquery
        SELECT *
        FROM `physionet-data.mimiciv_hosp.labevents` le
        LEFT JOIN (
          SELECT itemid, label FROM `physionet-data.mimiciv_hosp.d_labitems`
        )  dle2 ON dle2.itemid = le.itemid
        WHERE le.itemid IN (
          SELECT itemid FROM `physionet-data.mimiciv_hosp.d_labitems` WHERE category = "Blood Gas"
          )
    )
  )
  PIVOT (
    ANY_VALUE(value)
    FOR label IN (
      "pH",
      "pCO2",
      "pO2",
      "Oxygen Saturation" as SaO2,
      "Carboxyhemoglobin",
      "Methemoglobin",
      "Specimen Type" as SpecimenType
      )
  )
) pivoted
WHERE
subject_id IS NOT NULL
AND
hadm_id IS NOT NULL
AND
pH IS NOT NULL
AND
pCO2 IS NOT NULL
AND
SaO2 IS NOT NULL
AND
SpecimenType IS NOT NULL
AND
SpecimenType IN ("ART.")
ORDER BY subject_id ASC,  hadm_id ASC, charttime ASC

Query is running:   0%|          |

Downloading:   0%|          |

In [79]:
for col in ['charttime']:
  mimic_iv_abgs[col] = pd.to_datetime(mimic_iv_abgs[col], errors='coerce')

for col in ['subject_id', 'hadm_id',
            'pH','pCO2','pO2','SaO2'
            ]:
  mimic_iv_abgs[col] = pd.to_numeric(mimic_iv_abgs[col], errors='coerce', downcast = 'integer')

# filters to ensure all minimal ABG data (pH, pCO2, pO2, SaO2) are present
mimic_iv['abgs'] = mimic_iv_abgs[
    mimic_iv_abgs[[
        'subject_id', 'hadm_id', 'charttime',
        'pH', 'pCO2','pO2','SaO2']].notnull().all(axis=1)
]

mimic_iv_pat_abgs = mimic_iv['abgs'].merge(mimic_iv_pat,
                                           on='hadm_id',
                                           how='left')

mimic_iv_pat_abgs = mimic_iv_pat_abgs.groupby('subject_id').first()

In [80]:
dataframes[2] = mimic_iv_pat_abgs[['race_ethnicity', 'sex_female', 'admission_age']]

### 3. spo2 paired

In [81]:
%%bigquery mimic_iv_vitals

Select *
FROM (
----
  SELECT subject_id,hadm_id,stay_id, charttime, di2.label, valuenum
  FROM `physionet-data.mimiciv_icu.chartevents` ce
  LEFT JOIN `physionet-data.mimiciv_icu.d_items` di2 ON di2.itemid = ce.itemid
  WHERE ce.itemid IN (
    -- select temp, HR, BP, O2sat
      SELECT itemid
      FROM `physionet-data.mimiciv_icu.d_items` di
      WHERE (
          (linksto = 'chartevents')
          and
          di.category in ('Respiratory', 'Routine Vital Signs')
          and
          di.label in (
              'O2 saturation pulseoxymetry'
              )
          )
    )

  )
  PIVOT (
    avg(valuenum)
    FOR label IN (
            'O2 saturation pulseoxymetry'
      )

  )
ORDER BY subject_id ASC,  hadm_id ASC, charttime ASC

Query is running:   0%|          |

Downloading:   0%|          |

In [82]:
for col in ['charttime']:
  mimic_iv_vitals[col] = pd.to_datetime(mimic_iv_vitals[col], errors='coerce')

for col in ['subject_id', 'hadm_id', 'stay_id']:
  mimic_iv_vitals[col] = pd.to_numeric(mimic_iv_vitals[col], errors='coerce', downcast = 'integer')

mimic_iv['vitals'] = mimic_iv_vitals[
    # ensure there are identifiers
    mimic_iv_vitals[[
        'subject_id', 'hadm_id', 'charttime',
        ]].notnull().all(axis=1)
    # and at least one value in vitals
        &
            mimic_iv_vitals[[
            'O2 saturation pulseoxymetry'
        ]].notnull().any(axis=1)

]

mimic_iv['vitals'] = mimic_iv['vitals'].rename(columns={
    'O2 saturation pulseoxymetry':'SpO2',
})

mimic_iv['vitals_withSpO2'] = mimic_iv['vitals'][
    mimic_iv['vitals']['SpO2'].notnull()
].copy()

mimic_iv['vitals_withSpO2_SaO2'] = mimic_iv['vitals'][
    mimic_iv['vitals']['SpO2'].notnull()
    &
    # ensure vitals are just for patients with ABGs
    mimic_iv['vitals']['subject_id'].isin(mimic_iv['abgs']['subject_id'])
].copy()

In [83]:
mimic_iv['abgs'] = mimic_iv['abgs'].sort_values(by = [
      'charttime'
      ])
mimic_iv['vitals_withSpO2_SaO2'] = mimic_iv['vitals_withSpO2_SaO2'].sort_values(by = [
      'charttime'
    ])

mimic_iv_ABG_SpO2 = pd.merge_asof(
        left = mimic_iv['abgs'].rename(columns={'charttime':'charttime_abg'}),
        right = mimic_iv['vitals_withSpO2_SaO2'].rename(columns={'charttime':'charttime_vitals'}),
        left_by = 'hadm_id',
        right_by = 'hadm_id',
        left_on = 'charttime_abg',
        right_on = 'charttime_vitals',
        tolerance = pd.Timedelta(seconds = 5 * 60),
        direction = 'backward',
        suffixes = ('','_todrop'),
        allow_exact_matches = True
    )

mimic_iv_ABG_SpO2.drop(columns = [col for col in mimic_iv_ABG_SpO2.columns if '_todrop' in col], inplace=True)
mimic_iv_ABG_SpO2 = mimic_iv_ABG_SpO2[mimic_iv_ABG_SpO2['charttime_vitals'].notnull()]

cols = mimic_iv_ABG_SpO2.columns.tolist()
cols.insert(2, cols.pop(cols.index('stay_id')))
cols.insert(8, cols.pop(cols.index('SpO2')))
mimic_iv_ABG_SpO2 = mimic_iv_ABG_SpO2[cols]

mimic_iv_ABG_SpO2 = mimic_iv_ABG_SpO2.drop("SpecimenType", axis=1)
mimic_iv_ABG_SpO2['delta_SpO2'] =  (mimic_iv_ABG_SpO2['charttime_vitals'] - mimic_iv_ABG_SpO2['charttime_abg']) \
                                   .dt.total_seconds() / 60

mimic_iv_ABG_SpO2 = mimic_iv_ABG_SpO2.rename(columns={"charttime_abg": "SaO2_timestamp",
                                                      "charttime_vitals": "SpO2_timestamp",
                                                      })


with pd.option_context('display.max_columns',None):
  display(
    mimic_iv_ABG_SpO2.head(2)
  )

mimic_iv['SpO2_SaO2_pairs'] = mimic_iv_ABG_SpO2

del mimic_iv_ABG_SpO2

,subject_id,hadm_id,stay_id,SaO2_timestamp,pH,pCO2,pO2,SaO2,SpO2,Carboxyhemoglobin,Methemoglobin,SpO2_timestamp,delta_SpO2
19,16284044,23864737,36817845,2110-01-13 16:01:00,7.33,39.0,129.0,98.0,100.0,None,None,2110-01-13 16:00:00,-1.0
31,13201095,28453791,39953418,2110-01-19 20:53:00,7.50,27.0,140.0,98.0,99.0,None,None,2110-01-19 20:53:00,0.0


In [84]:
mimic_iv_pairs_pats = mimic_iv['SpO2_SaO2_pairs'].merge(mimic_iv_pat,
                                                                   on="subject_id",
                                                                   how="left")
mimic_iv_pairs_pats = mimic_iv_pairs_pats.groupby('subject_id').first()

In [85]:
dataframes[3] = mimic_iv_pairs_pats[['race_ethnicity', 'sex_female', 'admission_age']]

### 4. spo2 >= 92

In [86]:
mimic_iv_final = mimic_iv['SpO2_SaO2_pairs'][mimic_iv['SpO2_SaO2_pairs'].SpO2 >= 92]

mimic_iv_final_pats = mimic_iv_final.merge(mimic_iv_pat,
                                           on="subject_id",
                                           how="left")

mimic_iv_final_pats = mimic_iv_final_pats.groupby('subject_id').first()

In [87]:
dataframes[4] = mimic_iv_final_pats[['race_ethnicity', 'sex_female', 'admission_age']]

## merge all dfs

In [88]:
all_dfs = pd.DataFrame()

for key, df in dataframes.items():
  df['cohort'] = key
  all_dfs = pd.concat([all_dfs, df], axis=0)

In [89]:
all_dfs.cohort.value_counts()

1    50920
2    17845
3     4783
4     4662
Name: cohort, dtype: int64

In [90]:
all_dfs.race_ethnicity.fillna('Unknown')

all_dfs.race_ethnicity = all_dfs.race_ethnicity.map({
    'American Indian / Alaska Native': 'Unknown',
    'More Than One Race': 'Unknown',
    'Native Hawaiian / Pacific Islander': 'Unknown',
    'White': 'White',
    'Asian': 'Asian',
    'Hispanic OR Latino': 'Hispanic OR Latino',
    'Black': 'Black',
    'Unknown': 'Unknown'
})

In [91]:
all_dfs = all_dfs[
    all_dfs[[
        'race_ethnicity', 'admission_age', 'sex_female'
        ]].notnull().all(axis=1)
].reset_index()

## get table ones

In [95]:
categ = ['race_ethnicity', 'sex_female']
nonnormal = ['admission_age']

table = TableOne(all_dfs,
                 columns=categ+nonnormal,
                 categorical=categ,
                 nonnormal=nonnormal,
                 overall=False,
                 smd=True,
                 missing=False,
                 groupby='cohort')

In [98]:
table

Grouped by cohort                                                                                                                  
                                                                 1                 2                 3                 4 SMD (2,4) SMD (1,2) SMD (3,4) SMD (1,4) SMD (2,3) SMD (1,3)
n                                                            50920             13645              4783              4662                                                            
race_ethnicity, n (%)         Asian                     1496 (2.9)         332 (2.4)         112 (2.3)         112 (2.4)     0.035     0.126     0.006     0.100     0.032     0.101
                              Black                     4640 (9.1)         838 (6.1)         320 (6.7)         311 (6.7)                                                            
                              Hispanic OR Latino        1783 (3.5)         424 (3.1)         161 (3.4)         159 (3.4)                                                            
                              Unknown                  8619 (16.9)       2579 (18.9)        866 (18.1)        838 (18.0)                                                            
                              White                   34382 (67.5)       9472 (69.4)       3324 (69.5)       3242 (69.5)                                                            
sex_female, n (%)             0                       28440 (55.9)       8570 (62.8)       3099 (64.8)       3023 (64.8)     0.042     0.142     0.001     0.185     0.041     0.183
                              1                       22480 (44.1)       5075 (37.2)       1684 (35.2)       1639 (35.2)                                                            
admission_age, median [Q1,Q3]                     66.0 [54.0,78.0]  67.0 [57.0,77.0]  68.0 [59.0,77.0]  68.0 [59.0,77.0]     0.065     0.087     0.002     0.148     0.062     0.146